# Parse Archived Statement of Votes, Orange County (CA)

## Requirements

In [163]:
#import sys
#!{sys.executable} -m pip install requests
#!{sys.executable} -m pip install beautifulsoup4

import requests
from bs4 import BeautifulSoup
import re
import zipfile
import os
import shutil
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import csv
import numpy as np

## Set URL parameters

### Define function for creating temp directory

In [164]:
def create_directory(path):
    try:
        # Create the directory if it doesn't exist
        os.makedirs(path, exist_ok=True)
    except Exception as e:
        print(f"Failed to create directory '{path}'. Reason: {e}")

### Define incoming and temp paths

In [165]:
# URL to scrape
url = 'https://www.ocvote.gov/data/election-results-archives/archived-statement-of-votes'

# URL prefix
url_prefix = 'https://www.ocvote.gov'

# Local destination
local_prefix = '/Users/alisonpitt/Documents/Data Sets/oc_vote/'
create_directory(local_prefix)

## Fetch and parse index page contents

In [166]:
# Send a GET request to fetch the content of the page
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')
section = soup.find(class_="page__body")
list_of_values = section.find_all('p')

## Parse site for headings and links

In [167]:
# List to store the results
link_list = []

# Initialize the current election, link name and file name
current_election = ''
link = ''
dl_filename = ''

# Initialize list index
index = 0

# Loop through all <p> elements
for element in list_of_values:
    # Check if the element has a hyperlink
    if 'a href=' in str(element):
        link_type = element.string
        link = url_prefix + element.find('a')['href']
        dl_filename = link.rsplit('/', 1)[-1]
        # Only record the link if it's a zip file (aka the plain-text extract)
        if dl_filename[-3:] == 'zip':
            link_list.append((index, current_election, link_type, link, dl_filename))
            index = index + 1
    
    # Corrects for 2014-10-07 election where Statement of Votes has no link
    elif current_election == 'October 7, 2014 Yorba Linda Special Recall Election Statement of Votes' and element.string == 'Statement of Votes':
        pass

    # Otherwise
    else:
        current_election = element.string

link_list

[(0,
  'June 4, 2024 City of Anaheim District 3 Special Recall Election',
  'Plain-text extract for data analysis (media.zip).',
  'https://www.ocvote.gov/sites/default/files/elections/anarcl2024/results/media.zip',
  'media.zip'),
 (1,
  'March 5, 2024\xa0Presidential Primary Election',
  'Plain-text extract for data analysis (media.zip).',
  'https://www.ocvote.gov/fileadmin/live/PRI2024/media.zip',
  'media.zip'),
 (2,
  'November 14, 2023 City of Santa Ana Special Recall Election',
  'Plain-text extract for data analysis (media.zip).',
  'https://www.ocvote.gov/fileadmin/live/sarcl2023/media.zip',
  'media.zip'),
 (3,
  'October\xa03, 2023 City of Anaheim Special Municipal Election',
  'Plain-text extract for data analysis (media.zip).',
  'https://www.ocvote.gov/fileadmin/live/2023ANA/media.zip',
  'media.zip'),
 (4,
  'January 31, 2023 City of Seal Beach Districts 3 and 5 Municipal Run-Off Election',
  'Plain-text extract for data analysis (media.zip).',
  'https://www.ocvote.gov

## Download files one by one and append to a dataframe

To do this:
- Set up the file for download
- Download the remote zip file
- Extract zip file (throw error if more than one file)
- Union with full dataframe
- Delete temp files

### Define functions for parsing and appending

In [168]:
def delete_file(path):
    try:
        # Delete the zip file if it exists
        os.remove(path)
    except Exception as e:
        print(f"Failed to delete zip file '{path}'. Reason: {e}")

def is_first_row_value(file_path, value):
    with open(file_path, 'r', encoding='latin-1') as file:
        reader = csv.reader(file)
        first_row = next(reader)  # Get the first row
        return value in first_row

def detect_delimiter(file_path):
    comma_count = 0
    tab_count = 0
    
    with open(file_path, 'r', encoding='latin-1') as file:
        # Read the first 3 lines
        for _ in range(3):
            line = file.readline()
            if not line:
                break
            # Update the counts
            comma_count += line.count(',')
            tab_count += line.count('\t')

    # Determine the delimiter
    if comma_count > tab_count:
        delimiter = ','
    elif tab_count > comma_count:
        delimiter = '\t'
    else:
        delimiter = None  # Could not determine

    return delimiter

### Download all files and append to a new dataframe

In [169]:
full_df = pd.DataFrame()

for item in link_list:
    #print(f'--- Iteration {item[0]} ---')
    # Set up the file for download
    file_url = item[3] # Specify the URL of the file to download

    # Define the temp file
    local_tempfile = local_prefix + 'temp.zip'

    # Send a GET request to the URL
    dl_file = requests.get(file_url)

    # Write the content of the response to a local file
    with open(local_tempfile, 'wb') as file:
        file.write(dl_file.content)

    #print(f'File {file_url} was written to {local_tempfile}')

    # Extract the zip file
    with zipfile.ZipFile(local_tempfile, 'r') as zip_ref:
        temp_filenames = zip_ref.namelist()
        zip_ref.extractall(local_prefix)
    
    # Read all extracted files into a pandas dataframe
    for name in temp_filenames:
        # Check for delimiter changes and whether to skip first row
        # Detect delimiter
        detected_delimiter = detect_delimiter(local_prefix + name)
        if detected_delimiter in [',', '\t']:
            # Check if '#FormatVersion 1' is in the first row
            if is_first_row_value(local_prefix + name, '#FormatVersion 1'):
                df = pd.read_csv(local_prefix + name, delimiter=detected_delimiter, skiprows=1, encoding='latin-1')
            else:
                df = pd.read_csv(local_prefix + name, delimiter=detected_delimiter, encoding='latin-1')
        else:
            print(f'Delimiter unknown. File {name} from {item[1]} was skipped.')
        df['Election ID'] = item[0]
        df['Election Name'] = item[1]

        # Identify voters vs total voters column names
        #columns_with_voters = df.columns[df.columns.str.contains('voters', case=False)]
        #print(columns_with_voters)

        # Identify ballots vs total ballots column names
        #columns_with_ballots = df.columns[df.columns.str.contains('ballots', case=False)]
        #print(columns_with_ballots)

        # Identify votes vs total votes column names
        #columns_with_votes = df.columns[df.columns.str.contains('votes', case=False)]
        #print(columns_with_votes)
        
        full_df = pd.concat([df, full_df], ignore_index=True)

    # Delete the zip file
    delete_file(local_tempfile)

    # Delete the extracted file
    for name in temp_filenames:
        delete_file(local_prefix + name)

In [170]:
print(full_df)

         Precinct_Name  Split_Name  precinct_splitId  Reg_voters  Ballots  \
0               2001.0         NaN           28135.0       794.0    534.0   
1               2001.0         NaN           28135.0       794.0    534.0   
2               2001.0         NaN           28135.0       794.0    534.0   
3               2001.0         NaN           28135.0       794.0    534.0   
4               2001.0         NaN           28135.0       794.0    534.0   
...                ...         ...               ...         ...      ...   
2717209            NaN         NaN               NaN         NaN      NaN   
2717210            NaN         NaN               NaN         NaN      NaN   
2717211            NaN         NaN               NaN         NaN      NaN   
2717212            NaN         NaN               NaN         NaN      NaN   
2717213            NaN         NaN               NaN         NaN      NaN   

         Reporting  Contest_id                 Contest_title Contest_party 

## Transform the data

To do this:
- Split the dataframe into Precinct data and Election data
- Aggregate Precinct data to get to the right level of detail
- Additional cleaning as needed
- Export Precinct data and Election data to csv for Tableau ingestion

In [171]:
#print(full_df.head())
col_list = full_df.columns.to_list()
print(sorted(col_list))

['#Precinct', '.Precinct', 'AI Ballots Cast', 'AI Turnout Percentage', 'AI Voters', 'Absentee_votes', 'Ballots', 'Ballots Cast', 'Ballots_cast', 'Candidate_Type', 'Candidate_name', 'Choice ID', 'Choice Name', 'Choice Party', 'Choice_id', 'Choice_party', 'Contest ID', 'Contest Party', 'Contest Title', 'Contest_id', 'Contest_party', 'Contest_title', 'DEM Ballots Cast', 'DEM Turnout Percentage', 'DEM Voters', 'Early VC Invalid Votes', 'Early VC Overvotes', 'Early VC Undervotes', 'Early VC Votes', 'Early Vote Center Invalid Votes', 'Early Vote Center Overvotes', 'Early Vote Center Undervotes', 'Early Vote Center Votes', 'Early_votes', 'Election Day Vote Center Invalid Votes', 'Election Day Vote Center Overvotes', 'Election Day Vote Center Undervotes', 'Election Day Vote Center Votes', 'Election Days VC Invalid Votes', 'Election Days VC Overvotes', 'Election Days VC Undervotes', 'Election Days VC Votes', 'Election Days Vote Center Invalid Votes', 'Election Days Vote Center Overvotes', 'Elec

### Build Precinct data

In [172]:
precinct_data = pd.DataFrame()

# Coalesce dimensions
precinct_data['precinct'] = full_df.loc[:, ('#Precinct', '.Precinct', 'Precinct_Name')].bfill(axis=1).iloc[:, 0]
precinct_data['election_id'] = full_df['Election ID'].copy().astype(int)

In [173]:
# Add columns we'll use to calculate registered voters after aggregation
voters = pd.DataFrame()
filtered_columns_voters = [col for col in full_df.columns if 'voters' in col.lower()]
voters = full_df[filtered_columns_voters].copy()
precinct_data = precinct_data.join(voters)
print(precinct_data)

         precinct  election_id  Reg_voters  DEM Voters  REP Voters  AI Voters  \
0          2001.0           29       794.0         NaN         NaN        NaN   
1          2001.0           29       794.0         NaN         NaN        NaN   
2          2001.0           29       794.0         NaN         NaN        NaN   
3          2001.0           29       794.0         NaN         NaN        NaN   
4          2001.0           29       794.0         NaN         NaN        NaN   
...           ...          ...         ...         ...         ...        ...   
2717209    2813.0            0         NaN         NaN         NaN        NaN   
2717210    2834.0            0         NaN         NaN         NaN        NaN   
2717211    2834.0            0         NaN         NaN         NaN        NaN   
2717212   99999.0            0         NaN         NaN         NaN        NaN   
2717213   99999.0            0         NaN         NaN         NaN        NaN   

         GRN Voters  LIB Vo

In [174]:
# Add columns we'll use to calculate ballots cast after aggregation
ballots = pd.DataFrame()
filtered_columns_ballots = [col for col in full_df.columns if 'ballots' in col.lower()]
ballots = full_df[filtered_columns_ballots].copy()
precinct_data = precinct_data.join(ballots)
print(precinct_data)

         precinct  election_id  Reg_voters  DEM Voters  REP Voters  AI Voters  \
0          2001.0           29       794.0         NaN         NaN        NaN   
1          2001.0           29       794.0         NaN         NaN        NaN   
2          2001.0           29       794.0         NaN         NaN        NaN   
3          2001.0           29       794.0         NaN         NaN        NaN   
4          2001.0           29       794.0         NaN         NaN        NaN   
...           ...          ...         ...         ...         ...        ...   
2717209    2813.0            0         NaN         NaN         NaN        NaN   
2717210    2834.0            0         NaN         NaN         NaN        NaN   
2717211    2834.0            0         NaN         NaN         NaN        NaN   
2717212   99999.0            0         NaN         NaN         NaN        NaN   
2717213   99999.0            0         NaN         NaN         NaN        NaN   

         GRN Voters  LIB Vo

In [175]:
# Sum measures
## Voters
#mask_voters = full_df.columns.str.contains('Voters|voters')
#voters = full_df.loc[:, mask_voters]
#precinct_data['registered_voters'] = voters.sum(axis=1)

## Ballots
#mask_ballots = full_df.columns.str.contains('Ballots|ballots')
#ballots = full_df.loc[:, mask_ballots]
#precinct_data['ballots_cast'] = ballots.sum(axis=1)

### Build Election data

In [176]:
election_data = pd.DataFrame()

# Coalesce dimensions
election_data['precinct']       = precinct_data['precinct']
election_data['election_id']    = precinct_data['election_id']
election_data['election_name']  = full_df['Election Name'].copy()
election_data['contest_id']     = full_df['Contest ID'].combine_first(full_df['Contest_id'])
election_data['contest_party']  = full_df['Contest Party'].combine_first(full_df['Contest_party'])
election_data['contest_title']  = full_df['Contest Title'].combine_first(full_df['Contest_title'])
election_data['choice_id']      = full_df['Choice ID'].combine_first(full_df['Choice_id'])
election_data['choice_name']    = full_df['Choice Name'].combine_first(full_df['Candidate_name'])
election_data['choice_party']   = full_df['Choice Party'].combine_first(full_df['Choice_party'])
election_data['choice_type']    = full_df['Candidate_Type']

# ! BUG - this doubles votes where there are totals

In [177]:
# Sum measures
## Votes
#mask_votes = full_df.columns.str.contains('votes', case=False) & ~full_df.columns.str.contains('total', case=False)
#votes = full_df.loc[:, mask_votes]
#election_data['total_votes'] = votes.sum(axis=1)

In [178]:
# Sum measures
## Votes
## Later elections have a 'Total Votes' column, where earlier elections are split by absentee/early/election.
votes = pd.DataFrame()

# Create a temp df from columns containing 'votes'
mask_votes = full_df.columns.str.contains('votes', case=False)
votes = full_df.loc[:, mask_votes].copy()

# If 'Total Votes' = NULL then sum up the other columns
votes['Total Votes'] = votes.apply(
    lambda row: row.drop('Total Votes').sum() 
    if pd.isna(row['Total Votes']) else row['Total Votes'], 
    axis=1
)
print(votes)


         Absentee_votes  Early_votes  Election_Votes  Total Votes  \
0                   0.0          0.0            13.0         13.0   
1                 126.0          0.0           197.0        323.0   
2                   1.0          0.0             1.0          2.0   
3                 100.0          0.0            57.0        157.0   
4                   3.0          0.0             0.0          3.0   
...                 ...          ...             ...          ...   
2717209             NaN          NaN             NaN          0.0   
2717210             NaN          NaN             NaN         48.0   
2717211             NaN          NaN             NaN         36.0   
2717212             NaN          NaN             NaN          2.0   
2717213             NaN          NaN             NaN          2.0   

         Total Overvotes  Total Undervotes  Total Invalid Votes  \
0                    NaN               NaN                  NaN   
1                    NaN             

In [179]:
election_data['votes_cast'] = votes['Total Votes']

### Aggregate Precinct data to correct level of details (by precinct)

In [180]:
precinct_data = precinct_data.groupby(['precinct', 'election_id']).min().reset_index()

### Calculate registered voters

In [181]:
# Calculate registered voters (only do this after aggregation)
voters = pd.DataFrame()

# Create a temp df from columns containing 'voters'
mask_voters = precinct_data.columns.str.contains('voters', case=False)
voters = precinct_data.loc[:, mask_voters].copy()

# Coalesce 'Registered Voters' etc.
voters['registered_voters'] = voters['Registered Voters'].combine_first(voters['Reg_voters'])

# If registered_voters = NULL then sum up the other columns
voters['registered_voters'] = voters.apply(
    lambda row: row.drop(columns=['Registered Voters', 'Reg_voters', 'registered_voters']).sum()
    if pd.isna(row['registered_voters']) else row['registered_voters'],
    axis=1
)

In [182]:
# Join voters result back to precinct_data
precinct_data = precinct_data.join(voters['registered_voters'])

In [183]:
# Drop unnecessary voter columns
columns_to_drop = [col for col in precinct_data.columns if 'voters' in col.lower() and col != 'registered_voters']
precinct_data = precinct_data.drop(columns=columns_to_drop)

### Calculate ballots cast

In [184]:
# Calculate ballots cast (only do this after aggregation)
ballots = pd.DataFrame()

# Create a temp df from columns containing 'ballots'
mask_ballots = precinct_data.columns.str.contains('ballots', case=False)
ballots = precinct_data.loc[:, mask_ballots].copy()

In [185]:
# Coalesce 'Ballots Cast' etc.
ballots['ballots_cast'] = ballots.loc[:, ('Ballots Cast', 'Ballots_cast', 'Ballots')].bfill(axis=1).iloc[:, 0]

# If ballots_cast = NULL then sum up the other columns
ballots['ballots_cast'] = ballots.apply(
    lambda row: row.drop('ballots_cast').sum()
    if pd.isna(row['ballots_cast']) else row['ballots_cast'],
    axis=1
)

In [186]:
# Join ballots result back to precinct_data
precinct_data = precinct_data.join(ballots['ballots_cast'])

In [187]:
# Drop unnecessary ballot columns
columns_to_drop = [col for col in precinct_data.columns if 'ballots' in col.lower() and col != 'ballots_cast']
precinct_data = precinct_data.drop(columns=columns_to_drop)
print(precinct_data)

       precinct  election_id  registered_voters  ballots_cast
0        2001.0            0              594.0         101.0
1        2001.0            1              589.0          64.0
2        2001.0            3              568.0          74.0
3        2001.0            5              556.0         113.0
4        2001.0            6              550.0          71.0
...         ...          ...                ...           ...
25417   99999.0            2               52.0           7.0
25418   99999.0            3              423.0          46.0
25419   99999.0            4               90.0          10.0
25420   99999.0            5                1.0           1.0
25421   99999.0            6               19.0           4.0

[25422 rows x 4 columns]


### Set Precinct data types

In [188]:
# Assign data types
precinct_data = precinct_data.astype({
    'precinct': 'int'
    , 'election_id': 'int'
    , 'registered_voters': 'int'
    , 'ballots_cast': 'int'
})

### Set Election data types

In [189]:
# Assign data types
election_data = election_data.astype({
    'precinct': 'int'
    , 'election_id': 'int'
    , 'election_name': 'str'
    , 'contest_id': 'int'
    , 'contest_party': 'str'
    , 'contest_title': 'str'
    , 'choice_id': 'int'
    , 'choice_name': 'str'
    , 'choice_party': 'str'
    , 'choice_type': 'str'
    , 'votes_cast' : 'int'
})

In [190]:
precinct_rows, precinct_columns = precinct_data.shape
election_rows, election_columns = election_data.shape
print(f'Precinct data has {precinct_rows} rows and {precinct_columns} columns.')
print(f'Election data has {election_rows} rows and {election_columns} columns.')

Precinct data has 25422 rows and 4 columns.
Election data has 2717214 rows and 11 columns.


## Export to CSV

In [191]:
precinct_data.to_csv(local_prefix + 'precincts.csv', index=False)
election_data.to_csv(local_prefix + 'elections.csv', index=False)